# Proxy info 

* https://stormproxies.com/clients/residential

* Determine public IP: `dig +short myip.opendns.com @resolver1.opendns.com`
* Another awesome option: `curl checkip.amazonaws.com`

## Download a whole channel

check the .txt files for all the video IDs.

### mp4 and m4a codecs
TODO: I sould be able to specify the preffered codec as mp4.. that way there's less converting to mp4. 

```bash
But this only does BEST quality. We need to specify the height too.
original: -f "bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best"
kas modified: -f "bestvideo[ext=mp4,height<=360]+bestaudio[ext=m4a]/best[ext=mp4,height<=360]/best"
# close, but need more backup options if those formats aren't available
```

Is mp4 even the best option?

### parallel download (in command line)

```bash
parallel --line-buffer --jobs 8 yt-dlp -a yt_id_list.txt --download-archive archive.txt {}
```

Similar using Xargs
```bash
cat yt_id_list.txt | xargs -n 1 -P 8 yt-dlp --download-archive archive.txt

or

youtube-dl --get-id "$PLAYLIST_URL" \
  | xargs -I '{}' -P 5 youtube-dl 'https://youtube.com/watch?v={}'
```

## Download the YT-1B dataset

Just specify batch_file.

```bash
cd /home/kastan/thesis/video-pretrained-transformer/yt_download/YT_1B_VAL

yt-dlp -f 'bv*[height<=360]+ba/b[height<=480]' --download-archive yt_1b_val_download_record.txt --recode-video 'mp4' --batch-file ../__source/yttemporal1b_ids_val.txt -o '%(id)s_%(channel)s_%(view_count)s_%(title)s.%(ext)s'
```

In [ ]:
''' Download who challenls '''

# just find the channel URL ending in /videos
yt-dlp -f 'bv*[height<=360]+ba/b[height<=480]' --download-archive cinema_sins.txt --recode-video 'mp4' https://www.youtube.com/user/CinemaSins/videos -o '%(id)s_%(channel)s_%(view_count)s_%(title)s.%(ext)s'
yt-dlp -f 'bv*[height<=360]+ba/b[height<=480]' --download-archive SethsBikeHacks.txt --recode-video 'mp4' https://www.youtube.com/c/SethsBikeHacks/videos -o '%(id)s_%(channel)s_%(view_count)s_%(title)s.%(ext)s'
yt-dlp  'bv*[height<=360]+ba/b[height<=480]' --download-archive nilered.txt --recode-video 'mp4' https://www.youtube.com/c/NileRed/videos -o '%(id)s_%(channel)s_%(view_count)s_%(title)s.%(ext)s'
yt-dlp -f 'bv*[height<=360]+ba/b[height<=480]' --download-archive veritasium.txt https://www.youtube.com/c/veritasium/videos -o '%(id)s_%(channel)s_%(view_count)s_%(title)s.%(ext)s'
yt-dlp -f 'bv*[height<=360]+ba/b[height<=480]' --download-archive wendover.txt https://www.youtube.com/c/Wendoverproductions/videos -o '%(id)s_%(channel)s_%(view_count)s_%(title)s.%(ext)s'

### YT-dl python best options

https://github.com/yt-dlp/yt-dlp/blob/master/yt_dlp/YoutubeDL.py#L180

To split a file of video IDs into multiple files, use this command:

```bash
split --number=l/5 yt_id_list.txt outputprefix
```



-P "C:/MyVideos" -P "temp:tmp"
* set `-P "temp:tmp"` to an SSD
* set -P "C:/MyVideos" to the HDD final location
* --write-subs

* min_views
* download_archive
* proxy
* postprocessors
  * not worth it. just use FFMPEG in parallel later.
  * FFmpegVideoConvertorPP,
  * FFmpegPostProcessor,


#### Extract audio from all videos...
`for file in *.mkv; do ffmpeg -i "$file" -vn -sn -c copy "${file%.mkv}.mka; done`


In [ ]:
## NOT WORTH IT. 
"""
import yt_dlp

URLS = ['https://www.youtube.com/watch?v=BaW_jenozKc']

ydl_opts = {
    # 'format': 'm4a/bestaudio/best',
    # specify the order of preference using slashes.
    'format': 'bv*[height<=360]+ba/b[height<=480]', # b = best quality with BOTH video and audio.
    # ℹ️ See help(yt_dlp.postprocessor) for a list of available Postprocessors and their arguments
    # 'keepvideo': True,
    'postprocessors': [
        {  # Extract audio using ffmpeg
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'm4a',
        },
        {  # Extract audio using ffmpeg
        'key': 'FFmpegVideoConvertor',
        'preferedformat': 'webm',
        },
    ]
}


with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    error_code = ydl.download(URLS)
"""

# THE GOLDEN COMMAND

```bash
yt-dlp -f 'bv*[height<=360]+ba/b[height<=480]' \
  --download-archive yt_1b_train_download_record_{process_num}_of_9.txt \
  -P "C:/MyVideos" -P "temp:/tmp" \
  --batch-file ../__source/yttemporal1b_ids_val.txt \
  -o '%(id)s_%(channel)s_%(view_count)s_%(title)s.%(ext)s' \
  --min-views 1000 \
  --proxy tbd \
  --write-subs
```

1. split the csv into 10 chunks. 
2. output each to a seapara dir, for easy resuming. 

### Ray + subprocess for parallel bash

everything stored here: `/mnt/storage_hdd/thesis/yt_1b_dataset/yt_1b_train`

In [29]:
import ray 
import subprocess
import shlex

In [30]:
golden_command = """yt-dlp -f 'bv*[height<=360]+ba/b[height<=480]' \
  -P /mnt/storage_hdd/thesis/yt_1b_dataset/yt_1b_train/parallel_{process_num} \
  --batch-file /home/kastan/thesis/video-pretrained-transformer/yt_download/__source/train_id_collection/yt_1b_train_id_list_{process_num}.txt \
  --download-archive /mnt/storage_hdd/thesis/yt_1b_dataset/yt_1b_train/parallel_{process_num}/yt_1b_train_download_record_{process_num}.txt \
  -P 'temp:/tmp' \
  -N 5 \
  -o '%(id)s_%(channel)s_%(view_count)s_%(title)s.%(ext)s' \
  --min-views 500 \
  --proxy  \
  --write-subs"""
  
final_subprocess_commands = []
for i in range(10):
  final_subprocess_commands.append(shlex.split(golden_command.format(process_num=i)))
print(final_subprocess_commands[0])
print()
print(final_subprocess_commands[1])
print("Parallel downloads: ", len(final_subprocess_commands))

['yt-dlp', '-f', 'bv*[height<=360]+ba/b[height<=480]', '-P', '/mnt/storage_hdd/thesis/yt_1b_dataset/yt_1b_train/parallel_0', '--batch-file', '/home/kastan/thesis/video-pretrained-transformer/yt_download/__source/train_id_collection/yt_1b_train_id_list_0.txt', '--download-archive', '/mnt/storage_hdd/thesis/yt_1b_dataset/yt_1b_train/parallel_0/yt_1b_train_download_record_0.txt', '-P', 'temp:/tmp', '-N', '5', '-o', '%(id)s_%(channel)s_%(view_count)s_%(title)s.%(ext)s', '--min-views', '500', '--proxy', '69.30.217.114:19014', '--write-subs']

['yt-dlp', '-f', 'bv*[height<=360]+ba/b[height<=480]', '-P', '/mnt/storage_hdd/thesis/yt_1b_dataset/yt_1b_train/parallel_1', '--batch-file', '/home/kastan/thesis/video-pretrained-transformer/yt_download/__source/train_id_collection/yt_1b_train_id_list_1.txt', '--download-archive', '/mnt/storage_hdd/thesis/yt_1b_dataset/yt_1b_train/parallel_1/yt_1b_train_download_record_1.txt', '-P', 'temp:/tmp', '-N', '5', '-o', '%(id)s_%(channel)s_%(view_count)s_%(tit

In [ ]:
""" MAIN """
ray.shutdown()
ray.init()

@ray.remote
def persistent_download(subprocess_command):
  result = subprocess.run(subprocess_command)
  return result

# Launch four parallel square tasks.
futures = [persistent_download.remote(cmd) for cmd in final_subprocess_commands]

# Retrieve results.
all_results = ray.get(futures)

print(len(all_results))
print(all_results)

In [ ]:
''' TESTING WITH SUBPROCESS. WORKS ✅ (cant use check-output)'''

test_command = """yt-dlp -f 'bv*[height<=360]+ba/b[height<=480]' \
  -P /home/kastan/thesis/video-pretrained-transformer/test/ray_test \
  --batch-file /home/kastan/thesis/video-pretrained-transformer/test/ray_test/small_batch.txt \
  --download-archive /home/kastan/thesis/video-pretrained-transformer/test/ray_test/ray_test_download_record.txt \
  -P 'temp:/tmp' \
  -N 5 \
  -o '%(id)s_%(channel)s_%(view_count)s_%(title)s.%(ext)s' \
  --min-views 500 \
  --proxy 69.30.217.114:19014 \
  --write-subs"""
  
# single itme test
test_command = shlex.split(test_command)
result = subprocess.run(test_command)